# Hospital Readmission Exploratory Analysis

In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import kagglehub


In [ ]:
# Download latest version
path = kagglehub.dataset_download("vanpatangan/readmission-dataset")

print("Path to dataset files:", path)